## Adapted from Victor Zhou's CNN published at [this blog post](https://victorzhou.com/blog/keras-cnn-tutorial/).

In [1]:
import math
import mnist
import numpy as np

# Using tf.keras for Ghost Batch Norm capability.
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Nadam

import tensorflow as tf
#print(tf.version)
# Disable INFO and WARNING messages from TensorFlow.
# Our Keras version (2.2.4 / TF 1.15.0) throws deprecation warnings.
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
train_images = mnist.train_images() 
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

num_filters = 8
filter_size = 3
pool_size = 2

In [3]:
# Build the model.
batch_size = 100       # TODO: make this less arbitrary.
ghost_batch_size = 10  # TODO: make this less arbitrary.

model = Sequential([
    # The authors required ghost batch normalization in their experiments.
    # The papers they reference use normalization preceding each convolutional layer.
    BatchNormalization(virtual_batch_size=ghost_batch_size),
    Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(units=10, activation='softmax'),
])

In [4]:
# Adapted from https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
def step_lr_decay(epoch):
    # This starts with a learning rate higher than the default learning rate in
    # Vanilla SGD. It drops by a factor of 5 each step, and I chose to drop after
    # every other epoch to make training not take too long on a laptop.
    # Think of this as a half-life equation.
    initial_lrate=0.15
    drop=0.2
    epochs_drop=2.0
    current_rate = initial_lrate * math.pow(drop, math.floor(epoch / epochs_drop))
    print("Epoch %d learning rate: %f" % (epoch + 1, current_rate))
    return current_rate

In [5]:
# Compile the model.
epochs = 6            # Run enough epochs to test the training schedule.
learning_rate = 0.00  # Will be overrided by the scheduler.
decay_rate = 0.00     # Default decay rate.
momentum = 0.0        # Default momentum.


In [6]:
# Keras has no built in for this. 
# Code is from https://www.codementor.io/nitinsurya/how-to-re-initialize-keras-model-weights-et41zre2g
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [7]:
def hybrid_lr_decay(epoch):
    # Start off from the constant learning rate from the hybrid's initialization.
    initial_lrate=0.05
    drop=0.2
    epochs_drop=2.0
    current_rate = initial_lrate * math.pow(drop, math.floor(epoch / epochs_drop))
    print("Epoch %d learning rate: %f" % (epoch + 1, current_rate))
    return current_rate

hybrid_lr_scheduler = LearningRateScheduler(hybrid_lr_decay)

## Experiment 7: SGD with momentum, decaying learning rate.

In [ ]:
#reset_weights(model)
learning_rate = 0.07  # Set a new constant.
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Train the model.
decay_lr_scheduler = LearningRateScheduler(step_lr_decay)

exp7_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[decay_lr_scheduler]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1 learning rate: 0.150000
Epoch 1/6
60000/60000 [==============================] - 6s 100us/sample - loss: nan - acc: 0.1280 - val_loss: nan - val_acc: 0.0980
Epoch 2 learning rate: 0.150000
Epoch 2/6
60000/60000 [==============================] - 6s 98us/sample - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.0980
Epoch 3 learning rate: 0.030000
Epoch 3/6
60000/60000 [==============================] - 6s 98us/sample - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.0980
Epoch 4 learning rate: 0.030000
Epoch 4/6
60000/60000 [==============================] - 6s 99us/sample - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.0980
Epoch 5 learning rate: 0.006000
Epoch 5/6
 9300/60000 [===>..........................] - ETA: 5s - loss: nan - acc: 0.0955

In [ ]:
exp7_results.history

## Experiment 8: SGD with momentum, increasing batch size.

In [ ]:
reset_weights(model)
momentum = 0.1
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Just run each step independently. Keras has no callback for this.
exp8_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)
tmp_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels))
)

for i in tmp_results.history.keys():
    exp8_results.history[i].extend(tmp_results.history[i])

tmp_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 25,
    validation_data=(test_images, to_categorical(test_labels))
)

for i in tmp_results.history.keys():
    exp8_results.history[i].extend(tmp_results.history[i])


In [ ]:
exp8_results.history

## Experiment 9: SGD with momentum, hybrid.

In [ ]:
reset_weights(model)
learning_rate = 0.05  # Set a new constant.
momentum = 0.1
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Run a step, and then increase the batch size manually. The decay scheduler
# will decay the learning rate in the next step.
exp9_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)

tmp_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=4,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[hybrid_lr_scheduler]
)

for i in tmp_results.history.keys():
    if i == 'lr':
        continue
    exp9_results.history[i].extend(tmp_results.history[i])

In [ ]:
exp9_results.history

## Experiment 1-3 Plots

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
f_vanillasgd, a_vanillasgd = plt.subplots()
a_vanillasgd.plot(range(1,7), exp1_results.history["val_acc"], label="Decaying L-rate")
a_vanillasgd.plot(range(1,7), exp3_results.history["val_acc"], label="Hybrid")
a_vanillasgd.plot(range(1,7), exp2_results.history["val_acc"], label="Increasing Batch")

legend = a_vanillasgd.legend(loc='lower right', shadow=True, fontsize='medium')
a_vanillasgd.set(xlabel='epochs', ylabel='val acc', title='Vanilla SGD')
a_vanillasgd.grid()

f_vanillasgd.savefig("vanilla_sgd.png")

## Experiment 4-6 Plots

In [ ]:
f_adam, a_adam = plt.subplots()
a_adam.plot(range(1,7), exp4_results.history["val_acc"], label="Decaying L-rate")
a_adam.plot(range(1,7), exp6_results.history["val_acc"], label="Hybrid")
a_adam.plot(range(1,7), exp5_results.history["val_acc"], label="Increasing Batch")

legend = a_adam.legend(loc='lower right', shadow=True, fontsize='medium')
a_adam.set(xlabel='epochs', ylabel='val acc', title='Adam')
a_adam.grid()

f_adam.savefig("adam.png")

## Experiment 7-9 Plots

In [ ]:
f_sgdm, a_sgdm = plt.subplots()
a_sgdm.plot(range(1,7), exp7_results.history["val_acc"], label="Decaying L-rate")
a_sgdm.plot(range(1,7), exp9_results.history["val_acc"], label="Hybrid")
a_sgdm.plot(range(1,7), exp8_results.history["val_acc"], label="Increasing Batch")

legend = a_sgdm.legend(loc='lower right', shadow=True, fontsize='medium')
a_sgdm.set(xlabel='epochs', ylabel='val acc', title='SGD with momentum')
a_sgdm.grid()

f_sgdm.savefig("sgd_with_momentum.png")